<a href="https://colab.research.google.com/github/rupeshgyawali/federated-covid-xray-detection/blob/main/centralized/covid19_xray_detection_vgg16_backbone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download and unzip dataset

In [ ]:
# !cp /content/drive/MyDrive/MajorProject/COVID-19_Radiography_Dataset.zip /content/
!cp /content/drive/MyDrive/MajorProject/Dataset_1400.zip /content/

In [ ]:
# !gdown --id 1ZMgUQkwNqvMrZ8QaQmSbiDqXOWAewwou
# !unzip -q COVID-19_Radiography_Database.zip
# !gdown 10yfPVbWYhaZck9Z5ujE2IA2RmcEHadTP
# !unzip -q COVID-19_Radiography_Dataset.zip
!unzip -q Dataset_1400.zip -d Dataset_1400

### Import necessary libraries

In [ ]:
import numpy as np
import tensorflow as tf


### Prepare dataset

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
# dataset_path = '/content/COVID-19_Radiography_Dataset'
dataset_path = '/content/Dataset_1400'

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    dataset_path, seed=123, validation_split=0.2, subset="training", batch_size=batch_size, image_size=(img_height, img_width))
test_set = tf.keras.utils.image_dataset_from_directory(
    dataset_path, seed=123, validation_split=0.2, subset="validation", batch_size=batch_size, image_size=(img_height, img_width))

Found 5545 files belonging to 4 classes.
Using 4436 files for training.
Found 5545 files belonging to 4 classes.
Using 1109 files for validation.


### Model with vgg16 backbone

In [ ]:
# image_input = tf.keras.layers.Input(shape=(img_height, img_width, 3))
# data_augmentation = tf.keras.models.Sequential(
#   [
#     tf.keras.layers.RandomFlip("horizontal",
#                       input_shape=(img_height,
#                                   img_width,
#                                   3)),
#     tf.keras.layers.RandomRotation(0.1),
#     tf.keras.layers.RandomZoom(0.1),
#   ]
# )(image_input)

In [ ]:
vgg = tf.keras.applications.vgg16.VGG16(input_shape=[img_height,img_width]  + [3], weights='imagenet', include_top=False)#, input_tensor=data_augmentation)

In [ ]:
#dont train  existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
# x=tf.keras.layers.GlobalAveragePooling2D()(vgg.output)
x=tf.keras.layers.Flatten()(vgg.output)
x=tf.keras.layers.Dense(64, activation='relu')(x)
x=tf.keras.layers.Dropout(0.2)(x)
x=tf.keras.layers.Dense(128, activation='relu')(x)
prediction = tf.keras.layers.Dense(4,activation='softmax')(x)

In [ ]:
#create a model object
model = tf.keras.models.Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

### Train the model

In [ ]:
epochs = 50
tensorboard_callback = tf.keras.callbacks.TensorBoard(
            log_dir='/content/drive/MyDrive/MajorProject/logs/experiment5/centralized/')
csv_logger = tf.keras.callbacks.CSVLogger(
            '/content/drive/MyDrive/MajorProject/logs/experiment5/centralized/' + 'training.log', 
            append=True)

In [ ]:
history = model.fit(training_set, validation_data=test_set, epochs=epochs, callbacks=[tensorboard_callback, csv_logger])

Epoch 1/50
139/139 [==============================] - 28s 196ms/step - loss: 1.4309 - accuracy: 0.7000 - val_loss: 0.4722 - val_accuracy: 0.8287
Epoch 2/50
139/139 [==============================] - 28s 197ms/step - loss: 0.5325 - accuracy: 0.7969 - val_loss: 0.4959 - val_accuracy: 0.8440
Epoch 3/50
139/139 [==============================] - 29s 204ms/step - loss: 0.4132 - accuracy: 0.8381 - val_loss: 0.3919 - val_accuracy: 0.8918
Epoch 4/50
139/139 [==============================] - 29s 208ms/step - loss: 0.4094 - accuracy: 0.8573 - val_loss: 0.4436 - val_accuracy: 0.8494
Epoch 5/50
139/139 [==============================] - 30s 209ms/step - loss: 0.3392 - accuracy: 0.8711 - val_loss: 0.4145 - val_accuracy: 0.8810
Epoch 6/50
139/139 [==============================] - 30s 211ms/step - loss: 0.3062 - accuracy: 0.8765 - val_loss: 0.4934 - val_accuracy: 0.8656
Epoch 7/50
139/139 [==============================] - 30s 210ms/step - loss: 0.3422 - accuracy: 0.8785 - val_loss: 0.4253 - val_ac